<h1 align='center' style='color:purple'><b>CNN</b></h1>

In this project, I have used the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the Fashion MNIST dataset.

In [ ]:
#  Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. 
# The process of selecting the right set of hyperparameters for your machine learning (ML) application is called hyperparameter tuning or hypertuning.

!pip install keras-tuner

You should consider upgrading via the 'c:\users\rahul\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


### Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
# Import Dataset
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
# Load Training and Testing Data
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


In [ ]:
# Normalize pixel values, as the values are in range of 0-255, Divide it by 255 to convert the range to 0-1.
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
print('Shape of Training Data',train_images[0].shape)

Shape of Training Data (28, 28)


In [ ]:
# Reshape Data
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [ ]:
# Define a function build_model which returns a compiled model and uses hyperparameters you define inline to hypertune the model.

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')#output layer
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
# Import RandomSearch to perform parameter tuning
from kerastuner import RandomSearch

# Import HyperParameters, it serves as a hyerparameter container. A HyperParameters instance contains information about 
# both the search space and the current values of each hyperparameter. Hyperparameters can be defined inline with the 
# model-building code that uses them. This saves you from having to define the list of parameters every time.
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
# Run the hyperparameter search.
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1,batch_size=128)


Search: Running Trial #3

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |?                 
conv_1_kernel     |5                 |?                 
conv_2_filter     |32                |?                 
conv_2_kernel     |5                 |?                 
dense_1_units     |96                |?                 
learning_rate     |0.001             |?                 

Epoch 1/3
 22/422 [>.............................] - ETA: 4:05 - loss: 1.1244 - accuracy: 0.5849

In [ ]:
# Get Best model using .get_best_models attribute of tuner
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
# print best model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1769568   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,780,106
Trainable params: 1,780,106
Non-trainable params: 0
_________________________________________________________________
